In [3]:
import requests
import pandas as pd

# Get data from FPL API
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)
data = response.json()

# Export data to DataFrame
players = pd.DataFrame(data['elements'])
teams = pd.DataFrame(data['teams'])

# Get the required columns
df = players[['first_name', 'second_name', 'team', 'now_cost', 'total_points', 'minutes']].copy()
df['now_cost'] = df['now_cost'] / 10  # 50 -> 5.0M gibi

# Match team name
team_map = teams.set_index('id')['name'].to_dict()
df['team_name'] = df['team'].map(team_map)

# Combine first and last name
df['name'] = df['first_name'] + ' ' + df['second_name']

# Calculate value ratio
df['value_ratio'] = df['total_points'] / df['now_cost']

# Eliminate players with very few minutes (e.g. less than 500 minutes)
df_filtered = df[df['minutes'] > 500]

# Get the top 30 players
top_value_players = df_filtered.sort_values(by='value_ratio', ascending=False).head(30)

# Show only required columns
result = top_value_players[['name', 'team_name', 'now_cost', 'total_points', 'value_ratio']]

from tabulate import tabulate

# Get only required columns
table_data = top_value_players[['name', 'team_name', 'now_cost', 'total_points', 'value_ratio']].copy()

# Rename column names
table_data.columns = ['Player', 'Team', 'Value', 'Points', 'Points/Value']

# Print table
print(tabulate(table_data, headers='keys', tablefmt='fancy_grid', showindex=False, floatfmt=".2f"))


╒════════════════════════════╤════════════════╤═════════╤══════════╤════════════════╕
│ Player                     │ Team           │   Value │   Points │   Points/Value │
╞════════════════════════════╪════════════════╪═════════╪══════════╪════════════════╡
│ Matz Sels                  │ Nott'm Forest  │    5.00 │      150 │          30.00 │
├────────────────────────────┼────────────────┼─────────┼──────────┼────────────────┤
│ Bryan Mbeumo               │ Man Utd        │    8.00 │      236 │          29.50 │
├────────────────────────────┼────────────────┼─────────┼──────────┼────────────────┤
│ Jordan Pickford            │ Everton        │    5.50 │      158 │          28.73 │
├────────────────────────────┼────────────────┼─────────┼──────────┼────────────────┤
│ Dean Henderson             │ Crystal Palace │    5.00 │      135 │          27.00 │
├────────────────────────────┼────────────────┼─────────┼──────────┼────────────────┤
│ Chris Wood                 │ Nott'm Forest  │    7.5

In [13]:
# TOP 5 GOALKEEPERS

import requests
import pandas as pd
from tabulate import tabulate

# 1. Get data from FPL API
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)
data = response.json()

# 2. Export data to DataFrame
players_df = pd.DataFrame(data['elements'])
teams_df = pd.DataFrame(data['teams'])
positions_df = pd.DataFrame(data['element_types'])


# 3. Match team name
team_map = dict(zip(teams_df['id'], teams_df['name']))
position_map = {
    1: 'Goalkeeper',
    2: 'Defence',
    3: 'Midfielder',
    4: 'Forward'
}

players_df['team_name'] = players_df['team'].map(team_map)
players_df['position'] = players_df['element_type'].map(position_map)
players_df['now_cost'] = players_df['now_cost'] / 10  # 50 -> 5.0M gibi

# 4. Add player name and rating
players_df['name'] = players_df['first_name'] + ' ' + players_df['second_name']
players_df['value_ratio'] = players_df['total_points'] / (players_df['now_cost'])

# 5. Take the top 5 players from position
top_players_by_position = (
    players_df
    .sort_values(by='value_ratio', ascending=False)
    .groupby('position')
    .head(5)
)

# 6. Select and rename the required columns for the table

goalkeepers = top_players_by_position[top_players_by_position['position'] == 'Goalkeeper']
table_gk = goalkeepers[['name', 'team_name', 'position', 'now_cost', 'total_points', 'value_ratio']]
table_gk.columns = ['Player', 'Team', 'Position', 'Value', 'Points', 'Points/Value']

# 7. Tabloyu yazdır
print("\n💎 BEST P/P Goalkeepers by Position:\n")
print(tabulate(table_gk, headers='keys', tablefmt='fancy_grid', showindex=False, floatfmt=".2f"))




💎 BEST P/P Goalkeepers by Position:

╒══════════════════════╤════════════════╤════════════╤═════════╤══════════╤════════════════╕
│ Player               │ Team           │ Position   │   Value │   Points │   Points/Value │
╞══════════════════════╪════════════════╪════════════╪═════════╪══════════╪════════════════╡
│ Matz Sels            │ Nott'm Forest  │ Goalkeeper │    5.00 │      150 │          30.00 │
├──────────────────────┼────────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Jordan Pickford      │ Everton        │ Goalkeeper │    5.50 │      158 │          28.73 │
├──────────────────────┼────────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Dean Henderson       │ Crystal Palace │ Goalkeeper │    5.00 │      135 │          27.00 │
├──────────────────────┼────────────────┼────────────┼─────────┼──────────┼────────────────┤
│ David Raya Martín    │ Arsenal        │ Goalkeeper │    5.50 │      142 │          25.82 │
├──────────────────────┼────────

In [14]:
# TOP 5 DEFENCES
# 6. Select and rename the required columns for the table

defences = top_players_by_position[top_players_by_position['position'] == 'Defence']
table_df = defences[['name', 'team_name', 'position', 'now_cost', 'total_points', 'value_ratio']]
table_df.columns = ['Player', 'Team', 'Position', 'Value', 'Points', 'Points/Value']

# 7. Tabloyu yazdır
print("\n💎 BEST P/P Defences by Position:\n")
print(tabulate(table_df, headers='keys', tablefmt='fancy_grid', showindex=False, floatfmt=".2f"))



💎 BEST P/P Defences by Position:

╒════════════════════╤════════════════╤════════════╤═════════╤══════════╤════════════════╕
│ Player             │ Team           │ Position   │   Value │   Points │   Points/Value │
╞════════════════════╪════════════════╪════════════╪═════════╪══════════╪════════════════╡
│ Nikola Milenković  │ Nott'm Forest  │ Defence    │    5.50 │      145 │          26.36 │
├────────────────────┼────────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Aaron Wan-Bissaka  │ West Ham       │ Defence    │    4.50 │      118 │          26.22 │
├────────────────────┼────────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Daniel Muñoz Mejía │ Crystal Palace │ Defence    │    5.50 │      142 │          25.82 │
├────────────────────┼────────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Marc Guéhi         │ Crystal Palace │ Defence    │    4.50 │      116 │          25.78 │
├────────────────────┼────────────────┼────────────┼───

In [15]:
# TOP 5 MIDFIELDERS
# 6. Select and rename the required columns for the table

midfielders = top_players_by_position[top_players_by_position['position'] == 'Midfielder']
table_mf = midfielders[['name', 'team_name', 'position', 'now_cost', 'total_points', 'value_ratio']]
table_mf.columns = ['Player', 'Team', 'Position', 'Value', 'Points', 'Points/Value']

# 7. Tabloyu yazdır
print("\n💎 BEST P/P Midfielders by Position:\n")
print(tabulate(table_mf, headers='keys', tablefmt='fancy_grid', showindex=False, floatfmt=".2f"))


💎 BEST P/P Midfielders by Position:

╒═════════════════╤═════════════╤════════════╤═════════╤══════════╤════════════════╕
│ Player          │ Team        │ Position   │   Value │   Points │   Points/Value │
╞═════════════════╪═════════════╪════════════╪═════════╪══════════╪════════════════╡
│ Bryan Mbeumo    │ Man Utd     │ Midfielder │    8.00 │      236 │          29.50 │
├─────────────────┼─────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Jacob Murphy    │ Newcastle   │ Midfielder │    6.50 │      159 │          24.46 │
├─────────────────┼─────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Alex Iwobi      │ Fulham      │ Midfielder │    6.50 │      156 │          24.00 │
├─────────────────┼─────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Mohamed Salah   │ Liverpool   │ Midfielder │   14.50 │      344 │          23.72 │
├─────────────────┼─────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Antoine Semenyo │ Bournem

In [16]:
# TOP 5 FORWARDS
# 6. Select and rename the required columns for the table

forwards = top_players_by_position[top_players_by_position['position'] == 'Forward']
table_fw = forwards[['name', 'team_name', 'position', 'now_cost', 'total_points', 'value_ratio']]
table_fw.columns = ['Player', 'Team', 'Position', 'Value', 'Points', 'Points/Value']

# 7. Tabloyu yazdır
print("\n💎 BEST P/P Midfielders by Position:\n")
print(tabulate(table_fw, headers='keys', tablefmt='fancy_grid', showindex=False, floatfmt=".2f"))


💎 BEST P/P Midfielders by Position:

╒════════════════════════╤═══════════════╤════════════╤═════════╤══════════╤════════════════╕
│ Player                 │ Team          │ Position   │   Value │   Points │   Points/Value │
╞════════════════════════╪═══════════════╪════════════╪═════════╪══════════╪════════════════╡
│ Chris Wood             │ Nott'm Forest │ Forward    │    7.50 │      200 │          26.67 │
├────────────────────────┼───────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Yoane Wissa            │ Brentford     │ Forward    │    7.50 │      185 │          24.67 │
├────────────────────────┼───────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Jarrod Bowen           │ West Ham      │ Forward    │    8.00 │      193 │          24.12 │
├────────────────────────┼───────────────┼────────────┼─────────┼──────────┼────────────────┤
│ Raúl Jiménez Rodríguez │ Fulham        │ Forward    │    6.50 │      147 │          22.62 │
├─────────────────────